In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
from scipy import stats

# Dataset
battery_life = np.array([
    18.5, 20.1, 22.3, 19.8, 21.6, 23.2, 17.9, 25.1, 19.2, 20.9,
    18.8, 24.5, 21.2, 22.8, 20.5, 23.7, 19.6, 21.9, 26.4, 18.3
])

class BatteryAnalysisApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Battery Life Data Analysis")
        self.create_widgets()
        self.analyze_data()

    def create_widgets(self):
        # Frame for results
        self.frame = ttk.Frame(self.root, padding=10)
        self.frame.grid(row=0, column=0, sticky="nsew")

        # Mean and SD labels
        self.mean_label = ttk.Label(self.frame, text="Mean: ")
        self.mean_label.grid(row=0, column=0, sticky="w")
        self.sd_label = ttk.Label(self.frame, text="Standard Deviation: ")
        self.sd_label.grid(row=1, column=0, sticky="w")

        # Quartiles and IQR
        self.q1_label = ttk.Label(self.frame, text="Q1 (25th percentile): ")
        self.q1_label.grid(row=2, column=0, sticky="w")
        self.median_label = ttk.Label(self.frame, text="Median (Q2): ")
        self.median_label.grid(row=3, column=0, sticky="w")
        self.q3_label = ttk.Label(self.frame, text="Q3 (75th percentile): ")
        self.q3_label.grid(row=4, column=0, sticky="w")
        self.iqr_label = ttk.Label(self.frame, text="IQR: ")
        self.iqr_label.grid(row=5, column=0, sticky="w")

        # Percentiles and Deciles
        self.p90_label = ttk.Label(self.frame, text="90th Percentile: ")
        self.p90_label.grid(row=6, column=0, sticky="w")
        self.d4_label = ttk.Label(self.frame, text="D4 (40th percentile): ")
        self.d4_label.grid(row=7, column=0, sticky="w")

        # Outliers
        self.outliers_label = ttk.Label(self.frame, text="Outliers (IQR method): ")
        self.outliers_label.grid(row=8, column=0, sticky="w")

        # Z-scores Table Label
        ttk.Label(self.frame, text="Z-scores:").grid(row=9, column=0, sticky="w")

        # Text widget for z-scores
        self.zscores_text = tk.Text(self.frame, height=10, width=50)
        self.zscores_text.grid(row=10, column=0, columnspan=2, pady=5)

        # New battery input
        ttk.Label(self.frame, text="Enter new battery life (hours):").grid(row=11, column=0, sticky="w")
        self.new_battery_entry = ttk.Entry(self.frame)
        self.new_battery_entry.grid(row=11, column=1, sticky="w")

        self.check_button = ttk.Button(self.frame, text="Check Outlier", command=self.check_new_battery)
        self.check_button.grid(row=12, column=0, columnspan=2, pady=5)

        # Result for new battery
        self.new_battery_result = ttk.Label(self.frame, text="")
        self.new_battery_result.grid(row=13, column=0, columnspan=2, sticky="w")

    def analyze_data(self):
        # Calculate mean and std dev
        self.mean = np.mean(battery_life)
        self.sd = np.std(battery_life, ddof=1)  # sample std dev

        # Quartiles
        self.q1 = np.percentile(battery_life, 25)
        self.median = np.percentile(battery_life, 50)
        self.q3 = np.percentile(battery_life, 75)
        self.iqr = self.q3 - self.q1

        # Percentiles and deciles
        self.p90 = np.percentile(battery_life, 90)
        self.d4 = np.percentile(battery_life, 40)

        # Outlier bounds
        self.lower_bound = self.q1 - 1.5 * self.iqr
        self.upper_bound = self.q3 + 1.5 * self.iqr

        # Identify outliers using IQR method
        self.outliers = battery_life[(battery_life < self.lower_bound) | (battery_life > self.upper_bound)]

        # Calculate z-scores
        self.zscores = (battery_life - self.mean) / self.sd

        # Update GUI labels
        self.mean_label.config(text=f"Mean: {self.mean:.2f} hours")
        self.sd_label.config(text=f"Standard Deviation: {self.sd:.2f} hours")
        self.q1_label.config(text=f"Q1 (25th percentile): {self.q1:.2f} hours")
        self.median_label.config(text=f"Median (Q2): {self.median:.2f} hours")
        self.q3_label.config(text=f"Q3 (75th percentile): {self.q3:.2f} hours")
        self.iqr_label.config(text=f"IQR: {self.iqr:.2f} hours")
        self.p90_label.config(text=f"90th Percentile: {self.p90:.2f} hours")
        self.d4_label.config(text=f"D4 (40th percentile): {self.d4:.2f} hours")

        if len(self.outliers) == 0:
            self.outliers_label.config(text="Outliers (IQR method): None")
        else:
            outliers_str = ", ".join([f"{x:.2f}" for x in self.outliers])
            self.outliers_label.config(text=f"Outliers (IQR method): {outliers_str}")

        # Display z-scores
        self.zscores_text.delete(1.0, tk.END)
        for i, z in enumerate(self.zscores, start=1):
            outlier_mark = " *" if abs(z) > 2 else ""
            self.zscores_text.insert(tk.END, f"B{i}: {z:.2f}{outlier_mark}\n")

    def check_new_battery(self):
        try:
            new_val = float(self.new_battery_entry.get())
        except ValueError:
            messagebox.showerror("Invalid input", "Please enter a valid number for battery life.")
            return

        z_new = (new_val - self.mean) / self.sd
        is_outlier = abs(z_new) > 2

        result_text = f"New battery life: {new_val:.2f} hours\n"
        result_text += f"Z-score: {z_new:.2f}\n"
        if is_outlier:
            result_text += "This is considered an OUTLIER (|z| > 2)."
        else:
            result_text += "This is NOT an outlier."

        self.new_battery_result.config(text=result_text)


if __name__ == "__main__":
    root = tk.Tk()
    app = BatteryAnalysisApp(root)
    root.mainloop()
